In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
from multiprocessing import Pool, cpu_count, RLock
from scipy import stats
from statsmodels.tsa.stattools import adfuller

from financial_data_structure import resample_bar
from util import train_test_split, gen_dataset

In [29]:
data = pd.read_csv('BTCSPOT_300.csv', index_col=0, parse_dates=True)
data.shape

(489390, 5)